In [1]:
import json
from datetime import datetime

# filename = 'corona-out-2'
filename = 'corona-out-3'

In [2]:
def to_timestamp(val):
    dt = val.split(' ')
    st = f"{dt[5]}-{dt[1]}-{dt[2]} {dt[3]}"
    dt = datetime.strptime(st, "%Y-%b-%d %H:%M:%S")
    return int(dt.timestamp())

def to_dtstring(val):
    return datetime.fromtimestamp(val).strftime('%b %d, %Y - %X')

def get_hashtags(data):
    hashtags = []
    for item in data:
        hashtags.append(item['text'])
    return hashtags

In [3]:
# def add_user(user):
#     if (user['id'] not in users):
#         users[user['id']] = user

# def add_tweet(tweet, tweet_type):
#     if tweet_type=='tweet':
#         if tweet['id'] not in tweets:
#             tweets[tweet['id']] = tweet
#     elif tweet_type=='retweet':
#         if tweet['id'] not in retweets:
#             retweets[tweet['id']] = tweet

# def add_retweet(retweet):
#     if (retweet['id'] not in retweets):
#         retweets[retweet['id']] = retweet

def add_user(user):
    if (user['id_str'] not in users):
        new_user = {}
        
        new_user['id'] = user['id_str']
        new_user['name'] = user['name']
        new_user['screen_name'] = user['screen_name']
        new_user['location'] = user['location']
        new_user['url'] = user['url']
        new_user['description'] = user['description']
        new_user['verified'] = user['verified']
        new_user['followers_count'] = user['followers_count']
        new_user['friends_count'] = user['friends_count']
        new_user['listed_count'] = user['listed_count']
        new_user['favourites_count'] = user['favourites_count']
        new_user['statuses_count'] = user['statuses_count']
        new_user['created_at'] = to_timestamp(user['created_at'])
        
        users[user['id']] = new_user
        


def process_tweet(tweet):
    new_tweet = {}
        
    new_tweet['created_at'] = to_timestamp(tweet['created_at'])
    new_tweet['id'] = tweet['id_str']
    if tweet['truncated']:
        new_tweet['text'] = tweet['extended_tweet']['full_text']
        new_tweet['hashtags'] = get_hashtags(tweet['extended_tweet']['entities']['hashtags'])
    else:
        new_tweet['text'] = tweet['text']
        new_tweet['hashtags'] = get_hashtags(tweet['entities']['hashtags'])

    user = {}
    user['id'] = tweet['user']['id_str']
    user['name'] = tweet['user']['name']
    user['screen_name'] = tweet['user']['screen_name']
    user['verified'] = tweet['user']['verified']
    # updated
    user['friends_count'] = tweet['user']['friends_count']
    user['followers_count'] = tweet['user']['followers_count']
    user['description'] = tweet['user']['description']
    user['created_at'] = tweet['user']['created_at']
    new_tweet['user'] = user

    new_tweet['source'] = tweet['source']
    new_tweet['quote_count'] = tweet['quote_count']
    new_tweet['reply_count'] = tweet['reply_count']
    new_tweet['retweet_count'] = tweet['retweet_count']
    new_tweet['favorite_count'] = tweet['favorite_count']
    
    if 'retweeted_status' in tweet:
        new_tweet['retweeted_status'] = process_tweet(tweet['retweeted_status'])
        
    new_tweet['is_quote_status'] = tweet['is_quote_status']
    if tweet['is_quote_status'] and 'quoted_status' in tweet:
        new_tweet['quoted_status'] = process_tweet(tweet['quoted_status'])
    
    return new_tweet
        
def add_tweet(tweet, tweet_type):
    if (tweet['id_str'] not in tweets):
        
        new_tweet = process_tweet(tweet)
        
        if tweet_type=='tweet':
            tweets[tweet['id']] = new_tweet 
        elif tweet_type=='retweet':
            retweets[tweet['id']] = new_tweet 
        elif tweet_type=='quote':
            quotes[tweet['id']] = new_tweet

In [4]:
def process_data(data):
    if data['id_str'] in tweets:
        return
    
    add_user(data['user'])

    if data['text'].startswith('RT'):
        add_tweet(data, 'retweet')
        if 'retweeted_status' in data:
            process_data(data['retweeted_status'])

    else:
        add_tweet(data, 'tweet')

In [5]:
users = {}
tweets = {}
retweets = {}
quotes = {}

for line in open(filename, 'r'):
    if len(line.strip()) == 0:
        continue
        
#     try:
    data = json.loads(line)
    process_data(data)

#     except:
#         continue

In [6]:
# (7349, 11157, 15505)
# (10987, 11157, 18461)

len(tweets), len(retweets), len(users), len(quotes)

(50915, 61108, 88068, 0)

In [7]:
import pandas as pd 
userstable= pd.DataFrame(users.values())
# userstable

,id,name,screen_name,location,url,description,verified,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at
0,804046791348015107,Bi Sex Uau,B_King69,"Acre, Brasil",https://www.instagram.com/?hl=pt-br,se for da minha família já pode voltar daq mesmo,False,89,173,0,5446,4728,1480552668
1,2242948745,Thomas Krause,tho1965,None,None,Sportredakteur @nordkurier 🏃‍♂️🚴‍♂️⚽️,False,173,685,9,2184,1865,1387980813
2,908326492718764034,शचीन्द्र पाण्डेय,im_S_pandey,Amethi Uttar Pradesh,None,Official Twitter Handel Shachindra Pandey (@im...,False,2364,202,3,36747,49032,1505411286
3,207809313,BJP,BJP4India,"6-A, Deen Dayal Upadhyay Marg,",http://www.bjp.org,Official Twitter account of the Bharatiya Jana...,True,13483759,3,3019,0,200328,1288073947
4,2929344220,Ralf Schmitz,RusticusArat,🇩4790 Provinz,None,"BWLer,ex Offz,Tw meistens zwischen Tür & Angel...",False,778,733,2,32024,30551,1418915966
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88063,92819550,santosh,skpandey77,"Thanjavur, India",None,None,False,24,34,0,534,309,1259281922
88064,2279976427,Venugopi,Venu7630,None,None,None,False,3,35,0,83,42,1389084617
88065,4890378807,𝔻𝕒𝕪𝕤𝕖 gab.ai/AvelarDayse,AvelarDayse,Brazil,None,#JesusChrist❤︎︎ #QAnonPsyOp #StopZion࿕ #BanIsl...,False,3020,2306,2,29381,12704,1454999905
88066,872453029,R.A.shah,Rashah007,"Lucknow, India",None,None,False,881,1921,0,22823,9514,1349915039


In [25]:
retweets

{1254022772558368768: {'created_at': 1587831702,
  'id': '1254022772558368768',
  'text': 'RT @BJP4India: India’s war with Corona is ongoing.\n\nPlay your part and make sure no one around you starves.\n\nYour contribution to defeat hu…',
  'hashtags': [],
  'user': {'id': '908326492718764034',
   'name': 'शचीन्द्र पाण्डेय',
   'screen_name': 'im_S_pandey',
   'verified': False,
   'friends_count': 202,
   'followers_count': 2362,
   'description': 'Official Twitter Handel Shachindra Pandey (@im_s_pandey) BJP IT Department Amethi District\n Fallow by Narendra Modi \n (@narendramodi) ji',
   'created_at': 'Thu Sep 14 13:48:06 +0000 2017'},
  'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
  'quote_count': 0,
  'reply_count': 0,
  'retweet_count': 0,
  'favorite_count': 0,
  'retweeted_status': {'created_at': 1587814212,
   'id': '1253949413191344128',
   'text': 'India’s war with Corona is ongoing.\n\nPlay your part and make sure no one a

In [9]:
### UPLOADING DATA TO DB

In [10]:
from mysql.connector import Error
import mysql.connector as msql

try:
    conn = msql.connect(host='localhost', user='root', password='password')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE TwitterDB")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Database is created


In [11]:
try:
    conn = msql.connect(host='localhost', database='TwitterDB', user='root', password='password')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS users_data;')
        print('Creating table....')

        cursor.execute("CREATE TABLE users_data(id varchar(255),name varchar(255),screen_name varchar(255),location varchar(255),url varchar(255),description varchar(255),verified TINYINT,followers_count int,friends_count int,listed_count int,favourites_count int,statuses_count int,created_at int)")
        print("Table is created....")

        for i,row in userstable.iterrows():
            sql = "INSERT INTO TwitterDB.users_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))

        conn.commit()

except Error as e:
     print("Error while connecting to MySQL", e)

You're connected to database:  ('twitterdb',)
Creating table....
Table is created....


In [12]:
from pymongo import MongoClient
  
try:
    conn = MongoClient()
    print("Connected successfully!!!")
except:  
    print("Could not connect to MongoDB")
  
# database
db = conn.TwitterDB
collection = db.Tweets

# Insert Data
for key in tweets.keys():
    collection.insert_one(tweets[key])

# collection.insert_many(tweets)

Connected successfully!!!


In [13]:
# try:
#     conn = MongoClient()
#     print("Connected successfully!!!")
# except:  
#     print("Could not connect to MongoDB")
  
# # database
# db = conn.TwitterDB

collection = db.Retweets

# Insert Data
for key in retweets.keys():
    collection.insert_one(retweets[key])

In [14]:
def getUserByWord(word):
    conn = msql.connect(host='localhost', database='TwitterDB', user='root', password='Password')
    cursor = conn.cursor()
    query = f"SELECT * FROM users_data WHERE name LIKE '%{word}%' or screen_name LIKE '%{word}%'"
    cursor.execute(query)
    a=cursor.fetchall()
    cursor.close()
    conn.close()
    return a

In [15]:
def getTweetsbyString(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"text": {"$regex": word, "$options": "i"}}
    result = collection.find(query)
    a=[]
    for i in result:
        a.append(i)
        
    return a

def getTweetsbyHashtag(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"hashtags": {"$regex": f"{word}", "$options": "i"}}
    result = collection.find(query)
    a=[]
    for i in result:
        a.append(i)
        
    return a

def getTweetsbyUserID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"user.id": word}
    result = collection.find(query)
    a=[]
    for i in result:
        a.append(i)
        
    return a

def getRetweetsbyUserID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Retweets"]

    query = {"user.id": word}
    result = collection.find(query)
    a=[]
    for i in result:
        a.append(i)
        
    return a

def getRetweetsByOriginalTweetID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Retweets"]

    query = {"retweeted_status.id": word}
    result = collection.find(query)
    a=[]
    for i in result:
        a.append(i)
        
    return a